In [165]:
# PythonSim imports
from pythonSim import SimFunctions, SimClasses
from model_parameters import ModelParameters
from simulation_configuration import SimulationConfiguration

from patients.patient import Patient
from patients.leukemia_patient import LeukemiaPatient
from patients.transplant_patient import TransplantPatient
from patients.other_patient import OtherPatient

from utils import assign_nurse_station, all_left_department

# Python package imports
import math
import pandas as pd
import numpy as np
import scipy.stats as scs

# Progress bar imports
from tqdm.notebook import tqdm_notebook
import time
import logging
import os

In [166]:
# Initializing event calendar
Calendar = SimClasses.EventCalendar()

In [167]:
model_parameters = ModelParameters()
simulation_configuration = SimulationConfiguration()

In [173]:
q_flow_station = SimClasses.Resource()
q_flow_station.SetUnits(1)
q_flow_station_queue = SimClasses.FIFOQueue()

secretary_station = SimClasses.Resource()
secretary_station.SetUnits(2)

secretary_station_queue = SimClasses.FIFOQueue()

nurse_station_1 = SimClasses.Resource()
nurse_station_1.SetUnits(1)
nurse_station_2 = SimClasses.Resource()
nurse_station_2.SetUnits(1)
nurse_station_3 = SimClasses.Resource()
nurse_station_3.SetUnits(1)
nurse_station_4 = SimClasses.Resource()
nurse_station_4.SetUnits(1)
nurse_station_5 = SimClasses.Resource()
nurse_station_5.SetUnits(1)
nurse_station_6 = SimClasses.Resource()
nurse_station_6.SetUnits(1)

nurse_station_1_queue = SimClasses.FIFOQueue()
nurse_station_2_queue = SimClasses.FIFOQueue()
nurse_station_3_queue = SimClasses.FIFOQueue()
nurse_station_4_queue = SimClasses.FIFOQueue()
nurse_station_5_queue = SimClasses.FIFOQueue()
nurse_station_6_queue = SimClasses.FIFOQueue()

leukemia_doctor_1 = SimClasses.Resource()
leukemia_doctor_1.SetUnits(1)
leukemia_doctor_2 = SimClasses.Resource()
leukemia_doctor_2.SetUnits(1)
transplant_doctor_1 = SimClasses.Resource()
transplant_doctor_1.SetUnits(1)
transplant_doctor_2 = SimClasses.Resource()
transplant_doctor_2.SetUnits(1)
transplant_doctor_3 = SimClasses.Resource()
transplant_doctor_3.SetUnits(1)

leukemia_doctor_1_queue = SimClasses.FIFOQueue()
leukemia_doctor_2_queue = SimClasses.FIFOQueue()
transplant_doctor_1_queue = SimClasses.FIFOQueue()
transplant_doctor_2_queue = SimClasses.FIFOQueue()
transplant_doctor_3_queue = SimClasses.FIFOQueue()

In [179]:
q_flow_station_wait_time = SimClasses.DTStat()

secretary_station_wait_time = SimClasses.DTStat()

nurse_station_1_wait_time = SimClasses.DTStat()
nurse_station_2_wait_time = SimClasses.DTStat()
nurse_station_3_wait_time = SimClasses.DTStat()
nurse_station_4_wait_time = SimClasses.DTStat()
nurse_station_5_wait_time = SimClasses.DTStat()
nurse_station_6_wait_time = SimClasses.DTStat()

leukemia_doctor_1_wait_time = SimClasses.DTStat()
leukemia_doctor_2_wait_time = SimClasses.DTStat()

transplant_doctor_1_wait_time = SimClasses.DTStat()
transplant_doctor_2_wait_time = SimClasses.DTStat()
transplant_doctor_3_wait_time = SimClasses.DTStat()

In [ ]:
leukemia_doctor_1_complex_patients_total_processing_time = SimClasses.DTStat()
leukemia_doctor_1_regular_patients_total_processing_time = SimClasses.DTStat()
leukemia_doctor_2_complex_patients_total_processing_time = SimClasses.DTStat()
leukemia_doctor_2_regular_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_1_complex_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_1_regular_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_2_complex_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_2_regular_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_3_complex_patients_total_processing_time = SimClasses.DTStat()
transplant_doctor_3_regular_patients_total_processing_time = SimClasses.DTStat()
other_patients_total_processing_time = SimClasses.DTStat()

In [184]:
q_flow_station_wait_time_avg = []

secretary_station_wait_time_avg = []

nurse_station_1_wait_time_avg = []
nurse_station_2_wait_time_avg = []
nurse_station_3_wait_time_avg = []
nurse_station_4_wait_time_avg = []
nurse_station_5_wait_time_avg = []
nurse_station_6_wait_time_avg = []

leukemia_doctor_1_wait_time_avg = []
leukemia_doctor_2_wait_time_avg = []

transplant_doctor_1_wait_time_avg = []
transplant_doctor_2_wait_time_avg = []
transplant_doctor_3_wait_time_avg = []

q_flow_station_queue_length_avg = []

secretary_station_queue_length_avg = []

nurse_station_1_queue_length_avg = []
nurse_station_2_queue_length_avg = []
nurse_station_3_queue_length_avg = []
nurse_station_4_queue_length_avg = []
nurse_station_5_queue_length_avg = []
nurse_station_6_queue_length_avg = []

leukemia_doctor_1_queue_length_avg = []
leukemia_doctor_2_queue_length_avg = []

transplant_doctor_1_queue_length_avg = []
transplant_doctor_2_queue_length_avg = []
transplant_doctor_3_queue_length_avg = []

q_flow_station_busy_avg = []

secretary_station_busy_avg = []

nurse_station_1_busy_avg = []
nurse_station_2_busy_avg = []
nurse_station_3_busy_avg = []
nurse_station_4_busy_avg = []
nurse_station_5_busy_avg = []
nurse_station_6_busy_avg = []

leukemia_doctor_1_busy_avg = []
leukemia_doctor_2_busy_avg = []

transplant_doctor_1_busy_avg = []
transplant_doctor_2_busy_avg = []
transplant_doctor_3_busy_avg = []

In [ ]:
q_flow_station_queue_length = []

secretary_station_queue_length = []

nurse_station_1_queue_length = []
nurse_station_2_queue_length = []
nurse_station_3_queue_length = []
nurse_station_4_queue_length = []
nurse_station_5_queue_length = []
nurse_station_6_queue_length = []

leukemia_doctor_1_queue_length = []
leukemia_doctor_2_queue_length = []

transplant_doctor_1_queue_length = []
transplant_doctor_2_queue_length = []
transplant_doctor_3_queue_length = []

In [ ]:
leukemia_doctor_1_complex_patients_total_processing_time_avg = []
leukemia_doctor_1_regular_patients_total_processing_time_avg = []
leukemia_doctor_2_complex_patients_total_processing_time_avg = []
leukemia_doctor_2_regular_patients_total_processing_time_avg = []
transplant_doctor_1_complex_patients_total_processing_time_avg = []
transplant_doctor_1_regular_patients_total_processing_time_avg = []
transplant_doctor_2_complex_patients_total_processing_time_avg = []
transplant_doctor_2_regular_patients_total_processing_time_avg = []
transplant_doctor_3_complex_patients_total_processing_time_avg = []
transplant_doctor_3_regular_patients_total_processing_time_avg = []
other_patients_total_processing_time_avg = []

### We assume here that all patients begin arriving at 08:00 (time 0 is 07:00 hence why we add 60)
#### Currently we assume that they are all summoned at 20 minute intervals for simplicity

In [ ]:
leukemia_doctor_1_number_of_patients = model_parameters.leukemia_doctor_1_number_of_regular_patients + model_parameters.leukemia_doctor_1_number_of_complex_patients
leukemia_doctor_1_patient_arrival_times = [60 + i*20 for i in range(leukemia_doctor_1_number_of_patients)]
leukemia_doctor_2_number_of_patients = model_parameters.leukemia_doctor_2_number_of_regular_patients + model_parameters.leukemia_doctor_2_number_of_complex_patients
leukemia_doctor_2_patient_arrival_times = [60 + i*20 for i in range(leukemia_doctor_2_number_of_patients)]

transplant_doctor_1_number_of_patients = model_parameters.transplant_doctor_1_number_of_regular_patients + model_parameters.transplant_doctor_1_number_of_complex_patients
transplant_doctor_1_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_1_number_of_patients)]
transplant_doctor_2_number_of_patients = model_parameters.transplant_doctor_2_number_of_regular_patients + model_parameters.transplant_doctor_2_number_of_complex_patients
transplant_doctor_2_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_2_number_of_patients)]
transplant_doctor_3_number_of_patients = model_parameters.transplant_doctor_3_number_of_regular_patients + model_parameters.transplant_doctor_3_number_of_complex_patients
transplant_doctor_3_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_3_number_of_patients)]

other_patients_arrival_times = [60 + i*10 for i in range(model_parameters.number_of_other_patients)]

In [ ]:
def create_patient_schedule(arrival_time: int):
    return {'arrival_time': arrival_time,
            'doctor_consultation_time': arrival_time + 120}

def initialize_patients(patient_arrival_times: list[int],
                        doctor_name: str,
                        calendar: SimClasses.EventCalendar):
    leukemia = doctor_name.contains('leukemia')
    transplant = doctor_name.contains('transplant')
    patients = []
    if leukemia:
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_leukemia

        if doctor_name.contains('1'):
            probability_of_complex_patient = model_parameters.leukemia_doctor_1_probability_of_complex_patient
        else:
            probability_of_complex_patient = model_parameters.leukemia_doctor_2_probability_of_complex_patient

        for arrival_time in patient_arrival_times:
            patients.append(LeukemiaPatient(create_patient_schedule(arrival_time),
                                            doctor_name,
                                            probability_of_complex_patient,
                                            probability_of_visiting_nurse))
    elif transplant:
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_transplant
        if doctor_name.contains('1'):
            probability_of_complex_patient = model_parameters.transplant_doctor_1_probability_of_complex_patient
        elif doctor_name.contains('2'):
            probability_of_complex_patient = model_parameters.transplant_doctor_2_probability_of_complex_patient
        else:
            probability_of_complex_patient = model_parameters.transplant_doctor_3_probability_of_complex_patient

        for arrival_time in patient_arrival_times:
            patients.append(TransplantPatient(create_patient_schedule(arrival_time),
                                              doctor_name,
                                              probability_of_complex_patient,
                                              probability_of_visiting_nurse))
    else: #other
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_other
        probability_of_complex_patient = model_parameters.probability_of_complex_other_patient

        for arrival_time in patient_arrival_times:
            patients.append(OtherPatient(create_patient_schedule(arrival_time),
                                         doctor_name,
                                         probability_of_complex_patient,
                                         probability_of_visiting_nurse))
    
    for patient in patients:
        patient.schedule_arrival(calendar)
        
    return patients

In [191]:
def q_flow_station_start_of_waiting(new_patient : Patient):
    q_flow_station_queue.Add(new_patient)
    new_patient.enter_q_flow_queue()
    if q_flow_station.CurrentNumBusy < q_flow_station.NumberOfUnits:
        q_flow_station.Seize(1)
        next_patient = q_flow_station_queue.Remove()
        q_flow_station_wait_time.Record(SimClasses.Clock - next_patient.enter_q_flow_queue_time)
        SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_start", 0, next_patient)

In [192]:
def q_flow_station_service_start(new_patient : Patient):
    q_flow_service_duration = np.random.exponential(model_parameters.q_flow_mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_end", q_flow_service_duration, new_patient)

In [ ]:
def q_flow_station_service_end(new_patient: Patient):
    new_patient.enter_secretary_queue_time = SimClasses.Clock
    SimFunctions.SchedulePlus(Calendar, "secretary_station_start_of_waiting", 0, new_patient)

    q_flow_station.Free(1)
    if q_flow_station_queue.NumQueue() > 0:
        next_patient = q_flow_station_queue.Remove()
        q_flow_station_wait_time.Record(SimClasses.Clock - next_patient.enter_q_flow_queue_time)
        SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_start", 0, next_patient)

In [ ]:
def secretary_station_start_of_waiting(new_patient: Patient):
    secretary_station_queue.Add(new_patient)
    new_patient.enter_secretary_queue()
    if secretary_station.CurrentNumBusy < secretary_station.NumberOfUnits:
        secretary_station.Seize(1)
        next_patient = secretary_station_queue.Remove()
        secretary_station_wait_time.Record(SimClasses.Clock - next_patient.enter_secretary_queue_time)
        SimFunctions.SchedulePlus(Calendar, "secretary_station_service_start", 0, next_patient)

In [ ]:
def secretary_station_service_start(new_patient: Patient):
    secretary_station_service_duration = np.random.exponential(model_parameters.secretary_mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "secretary_station_service_end", secretary_station_service_duration, new_patient)

In [ ]:
def secretary_station_service_end(new_patient: Patient):
    patient_type = new_patient.get_type()
    has_nurse_appointment = new_patient.visits_nurse
    if has_nurse_appointment:
        nurse_station = assign_nurse_station(patient_type, model_parameters)
        SimFunctions.SchedulePlus(Calendar, f'{nurse_station}_start_of_waiting', 0, new_patient)
    else:
        SimFunctions.SchedulePlus(Calendar, f'{new_patient.doctor_name}_start_of_waiting', 0, new_patient)
    
    secretary_station.Free(1)
    if secretary_station_queue.NumQueue() > 0:
        next_patient = secretary_station_queue.Remove()
        secretary_station_wait_time.Record(SimClasses.Clock - next_patient.enter_secretary_queue_time)
        SimFunctions.SchedulePlus(Calendar, "secretary_station_service_start", 0, next_patient)

In [ ]:
def nurse_station_1_start_of_waiting(new_patient: Patient):
    nurse_station_1_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_1.CurrentNumBusy < nurse_station_1.NumberOfUnits:
        nurse_station_1.Seize(1)
        next_patient = nurse_station_1_queue.Remove()
        nurse_station_1_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_1_service_start", 0, next_patient)

def nurse_station_2_start_of_waiting(new_patient: Patient):
    nurse_station_2_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_2.CurrentNumBusy < nurse_station_2.NumberOfUnits:
        nurse_station_2.Seize(1)
        next_patient = nurse_station_2_queue.Remove()
        nurse_station_2_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_2_service_start", 0, next_patient)

def nurse_station_3_start_of_waiting(new_patient: Patient):
    nurse_station_3_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_3.CurrentNumBusy < nurse_station_3.NumberOfUnits:
        nurse_station_3.Seize(1)
        next_patient = nurse_station_3_queue.Remove()
        nurse_station_3_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_3_service_start", 0, next_patient)

def nurse_station_4_start_of_waiting(new_patient: Patient):
    nurse_station_4_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_4.CurrentNumBusy < nurse_station_4.NumberOfUnits:
        nurse_station_4.Seize(1)
        next_patient = nurse_station_4_queue.Remove()
        nurse_station_4_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_4_service_start", 0, next_patient)

def nurse_station_5_start_of_waiting(new_patient: Patient):
    nurse_station_5_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_5.CurrentNumBusy < nurse_station_5.NumberOfUnits:
        nurse_station_5.Seize(1)
        next_patient = nurse_station_5_queue.Remove()
        nurse_station_5_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_5_service_start", 0, next_patient)

def nurse_station_6_start_of_waiting(new_patient: Patient):
    nurse_station_6_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_6.CurrentNumBusy < nurse_station_6.NumberOfUnits:
        nurse_station_6.Seize(1)
        next_patient = nurse_station_6_queue.Remove()
        nurse_station_6_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_6_service_start", 0, next_patient)

In [ ]:
def nurse_station_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_1_service_end", service_duration, new_patient)

def nurse_station_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_2_service_end", service_duration, new_patient)

def nurse_station_3_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_3_service_end", service_duration, new_patient)

def nurse_station_4_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_4_service_end", service_duration, new_patient)

def nurse_station_5_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_5_service_end", service_duration, new_patient)

def nurse_station_6_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_6_service_end", service_duration, new_patient)

In [ ]:
def nurse_station_1_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_2_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_3_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_4_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_5_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_6_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

In [ ]:
def leukemia_doctor_1_start_of_waiting(new_patient: Patient):
    leukemia_doctor_1_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_start", 0, new_patient)
    if leukemia_doctor_1.CurrentNumBusy < leukemia_doctor_1.NumberOfUnits:
        leukemia_doctor_1.Seize(1)
        next_patient = leukemia_doctor_1_queue.Remove()
        leukemia_doctor_1_wait_time.Record(SimClasses.Clock - next_patient.enter_leukemia_doctor_1_queue_time)
        SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_start", 0, next_patient)

def leukemia_doctor_2_start_of_waiting(new_patient: Patient):
    leukemia_doctor_2_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_start", 0, new_patient)
    if leukemia_doctor_2.CurrentNumBusy < leukemia_doctor_2.NumberOfUnits:
        leukemia_doctor_2.Seize(1)
        next_patient = leukemia_doctor_2_queue.Remove()
        leukemia_doctor_2_wait_time.Record(SimClasses.Clock - next_patient.enter_leukemia_doctor_2_queue_time)
        SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_start", 0, next_patient)

def transplant_doctor_1_start_of_waiting(new_patient: Patient):
    transplant_doctor_1_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_start", 0, new_patient)
    if transplant_doctor_1.CurrentNumBusy < transplant_doctor_1.NumberOfUnits:
        transplant_doctor_1.Seize(1)
        next_patient = transplant_doctor_1_queue.Remove()
        transplant_doctor_1_wait_time.Record(SimClasses.Clock - next_patient.enter_transplant_doctor_1_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_start", 0, next_patient)

def transplant_doctor_2_start_of_waiting(new_patient: Patient):
    transplant_doctor_2_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_start", 0, new_patient)
    if transplant_doctor_2.CurrentNumBusy < transplant_doctor_2.NumberOfUnits:
        transplant_doctor_2.Seize(1)
        next_patient = transplant_doctor_2_queue.Remove()
        transplant_doctor_2_wait_time.Record(SimClasses.Clock - next_patient.enter_transplant_doctor_2_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_start", 0, next_patient)

def transplant_doctor_3_start_of_waiting(new_patient: Patient):
    transplant_doctor_3_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_start", 0, new_patient)
    if transplant_doctor_3.CurrentNumBusy < transplant_doctor_3.NumberOfUnits:
        transplant_doctor_3.Seize(1)
        next_patient = transplant_doctor_3_queue.Remove()
        transplant_doctor_3_wait_time.Record(SimClasses.Clock - next_patient.enter_transplant_doctor_3_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_start", 0, next_patient)

def other_doctor_start_of_waiting(new_patient: Patient):
    #other patients doctor treatment is not included in the model
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

In [ ]:
def leukemia_doctor_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.leukemia_doctor_1_mean_service_time_regular
    else:
        mean_service_time = model_parameters.leukemia_doctor_1_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_end", service_duration, new_patient)

def leukemia_doctor_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.leukemia_doctor_2_mean_service_time_regular
    else:
        mean_service_time = model_parameters.leukemia_doctor_2_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_end", service_duration, new_patient)

def transplant_doctor_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_1_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_1_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_end", service_duration, new_patient)

def transplant_doctor_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_2_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_2_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_end", service_duration, new_patient)

def transplant_doctor_3_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_3_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_3_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_end", service_duration, new_patient)

In [ ]:
def leukemia_doctor_1_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

def leukemia_doctor_2_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

def transplant_doctor_1_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

def transplant_doctor_2_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

def transplant_doctor_3_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

In [ ]:
def process_complete(new_patient: Patient):
    if new_patient.doctor_name == "leukemia_doctor_1":
        if new_patient.complexity_level == "complex":
            leukemia_doctor_1_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
        else:
            leukemia_doctor_1_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
    elif new_patient.doctor_name == "leukemia_doctor_2":
        if new_patient.complexity_level == "complex":
            leukemia_doctor_2_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
        else:
            leukemia_doctor_2_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
    elif new_patient.doctor_name == "transplant_doctor_1":
        if new_patient.complexity_level == "complex":
            transplant_doctor_1_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
        else:
            transplant_doctor_1_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
    elif new_patient.doctor_name == "transplant_doctor_2":
        if new_patient.complexity_level == "complex":
            transplant_doctor_2_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
        else:
            transplant_doctor_2_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
    elif new_patient.doctor_name == "transplant_doctor_3":
        if new_patient.complexity_level == "complex":
            transplant_doctor_3_complex_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
        else:
            transplant_doctor_3_regular_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
    elif new_patient.doctor_name == "other":
        other_patients_total_processing_time.Record(SimClasses.Clock - new_patient.getTime())
    else:
        raise ValueError(f"Invalid doctor name: {new_patient.doctor_name}")
    
    new_patient.end_visit()

In [ ]:
pbar_outer = tqdm_notebook(total=simulation_configuration.num_epochs, desc="Running Simulation")
for epoch in range(simulation_configuration.num_epochs):
    SimFunctions.SimFunctionsInit(Calendar)
    leukemia_doctor_1_patients = initialize_patients(leukemia_doctor_1_patient_arrival_times, "leukemia_doctor_1")
    leukemia_doctor_2_patients = initialize_patients(leukemia_doctor_2_patient_arrival_times, "leukemia_doctor_2")
    transplant_doctor_1_patients = initialize_patients(transplant_doctor_1_patient_arrival_times, "transplant_doctor_1")
    transplant_doctor_2_patients = initialize_patients(transplant_doctor_2_patient_arrival_times, "transplant_doctor_2")
    transplant_doctor_3_patients = initialize_patients(transplant_doctor_3_patient_arrival_times, "transplant_doctor_3")
    other_patients = initialize_patients(other_patients_arrival_times, "other")
    list_of_patients = [leukemia_doctor_1_patients, leukemia_doctor_2_patients, transplant_doctor_1_patients, transplant_doctor_2_patients, transplant_doctor_3_patients, other_patients]
    
    pbar_inner = tqdm_notebook(total=Calendar.N(), desc=f"Iteration {epoch + 1} in process", leave=False)
    while Calendar.N() > 0 and not all_left_department(list_of_patients):
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime

        if NextEvent.EventType == "q_flow_station_start_of_waiting":
            q_flow_station_queue_length.append(q_flow_station_queue.NumQueue())
            q_flow_station_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "q_flow_station_service_start":
            q_flow_station_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "q_flow_station_service_end":
            q_flow_station_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "secretary_station_start_of_waiting":
            secretary_station_queue_length.append(secretary_station_queue.NumQueue())
            secretary_station_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "secretary_station_service_start":
            secretary_station_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "secretary_station_service_end":
            secretary_station_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_1_start_of_waiting":
            nurse_station_1_queue_length.append(nurse_station_1_queue.NumQueue())
            nurse_station_1_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_1_service_start":
            nurse_station_1_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_1_service_end":
            nurse_station_1_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_2_start_of_waiting":
            nurse_station_2_queue_length.append(nurse_station_2_queue.NumQueue())
            nurse_station_2_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_2_service_start":
            nurse_station_2_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_2_service_end":
            nurse_station_2_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_3_start_of_waiting":
            nurse_station_3_queue_length.append(nurse_station_3_queue.NumQueue())
            nurse_station_3_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_3_service_start":
            nurse_station_3_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_3_service_end":
            nurse_station_3_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_4_start_of_waiting":
            nurse_station_4_queue_length.append(nurse_station_4_queue.NumQueue())
            nurse_station_4_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_4_service_start":
            nurse_station_4_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_4_service_end":
            nurse_station_4_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_5_start_of_waiting":
            nurse_station_5_queue_length.append(nurse_station_5_queue.NumQueue())
            nurse_station_5_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_5_service_start":
            nurse_station_5_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_5_service_end":
            nurse_station_5_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_6_start_of_waiting":
            nurse_station_6_queue_length.append(nurse_station_6_queue.NumQueue())
            nurse_station_6_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_6_service_start":
            nurse_station_6_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "nurse_station_6_service_end":
            nurse_station_6_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_1_start_of_waiting":
            leukemia_doctor_1_queue_length.append(leukemia_doctor_1_queue.NumQueue())
            leukemia_doctor_1_start_of_waiting(NextEvent.WhichObject)   

        elif NextEvent.EventType == "leukemia_doctor_1_service_start":
            leukemia_doctor_1_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_1_service_end":
            leukemia_doctor_1_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_2_start_of_waiting":
            leukemia_doctor_2_queue_length.append(leukemia_doctor_2_queue.NumQueue())
            leukemia_doctor_2_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_2_service_start":
            leukemia_doctor_2_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "leukemia_doctor_2_service_end":
            leukemia_doctor_2_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_1_start_of_waiting":
            transplant_doctor_1_queue_length.append(transplant_doctor_1_queue.NumQueue())
            transplant_doctor_1_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_1_service_start":
            transplant_doctor_1_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_1_service_end":
            transplant_doctor_1_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_2_start_of_waiting":
            transplant_doctor_2_queue_length.append(transplant_doctor_2_queue.NumQueue())
            transplant_doctor_2_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_2_service_start":
            transplant_doctor_2_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_2_service_end":
            transplant_doctor_2_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_3_start_of_waiting":
            transplant_doctor_3_queue_length.append(transplant_doctor_3_queue.NumQueue())
            transplant_doctor_3_start_of_waiting(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_3_service_start":
            transplant_doctor_3_service_start(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "transplant_doctor_3_service_end":
            transplant_doctor_3_service_end(NextEvent.WhichObject)
        
        elif NextEvent.EventType == "process_complete" or NextEvent.EventType == "other_doctor_start_of_waiting":
            #here the process ends for the other patients
            process_complete(NextEvent.WhichObject)
        else:
            logging.info(NextEvent.EventType)
            logging.error("Unknown event type")
            raise ValueError(f"Unknown event type: {NextEvent.EventType}")

        pbar_inner.update(1)
    pbar_inner.close()

    #storing statistics from all DTStat objects
    q_flow_station_wait_time_avg.append(q_flow_station_wait_time.Mean())
    secretary_station_wait_time_avg.append(secretary_station_wait_time.Mean())
    nurse_station_1_wait_time_avg.append(nurse_station_1_wait_time.Mean())
    nurse_station_2_wait_time_avg.append(nurse_station_2_wait_time.Mean())
    nurse_station_3_wait_time_avg.append(nurse_station_3_wait_time.Mean())
    nurse_station_4_wait_time_avg.append(nurse_station_4_wait_time.Mean())
    nurse_station_5_wait_time_avg.append(nurse_station_5_wait_time.Mean())
    nurse_station_6_wait_time_avg.append(nurse_station_6_wait_time.Mean())
    leukemia_doctor_1_wait_time_avg.append(leukemia_doctor_1_wait_time.Mean())
    leukemia_doctor_2_wait_time_avg.append(leukemia_doctor_2_wait_time.Mean())
    transplant_doctor_1_wait_time_avg.append(transplant_doctor_1_wait_time.Mean())
    transplant_doctor_2_wait_time_avg.append(transplant_doctor_2_wait_time.Mean())
    transplant_doctor_3_wait_time_avg.append(transplant_doctor_3_wait_time.Mean())

    leukemia_doctor_1_complex_patients_total_processing_time_avg.append(leukemia_doctor_1_complex_patients_total_processing_time.Mean())
    leukemia_doctor_1_regular_patients_total_processing_time_avg.append(leukemia_doctor_1_regular_patients_total_processing_time.Mean())
    leukemia_doctor_2_complex_patients_total_processing_time_avg.append(leukemia_doctor_2_complex_patients_total_processing_time.Mean())
    leukemia_doctor_2_regular_patients_total_processing_time_avg.append(leukemia_doctor_2_regular_patients_total_processing_time.Mean())
    transplant_doctor_1_complex_patients_total_processing_time_avg.append(transplant_doctor_1_complex_patients_total_processing_time.Mean())
    transplant_doctor_1_regular_patients_total_processing_time_avg.append(transplant_doctor_1_regular_patients_total_processing_time.Mean())
    transplant_doctor_2_complex_patients_total_processing_time_avg.append(transplant_doctor_2_complex_patients_total_processing_time.Mean())
    transplant_doctor_2_regular_patients_total_processing_time_avg.append(transplant_doctor_2_regular_patients_total_processing_time.Mean())
    transplant_doctor_3_complex_patients_total_processing_time_avg.append(transplant_doctor_3_complex_patients_total_processing_time.Mean())
    transplant_doctor_3_regular_patients_total_processing_time_avg.append(transplant_doctor_3_regular_patients_total_processing_time.Mean())
    other_patients_total_processing_time_avg.append(other_patients_total_processing_time.Mean())

    q_flow_station_queue_length_avg.append(q_flow_station_queue.Mean())
    secretary_station_queue_length_avg.append(secretary_station_queue.Mean())
    nurse_station_1_queue_length_avg.append(nurse_station_1_queue.Mean())
    nurse_station_2_queue_length_avg.append(nurse_station_2_queue.Mean())
    nurse_station_3_queue_length_avg.append(nurse_station_3_queue.Mean())
    nurse_station_4_queue_length_avg.append(nurse_station_4_queue.Mean())
    nurse_station_5_queue_length_avg.append(nurse_station_5_queue.Mean())
    nurse_station_6_queue_length_avg.append(nurse_station_6_queue.Mean())
    leukemia_doctor_1_queue_length_avg.append(leukemia_doctor_1_queue.Mean())
    leukemia_doctor_2_queue_length_avg.append(leukemia_doctor_2_queue.Mean())
    transplant_doctor_1_queue_length_avg.append(transplant_doctor_1_queue.Mean())
    transplant_doctor_2_queue_length_avg.append(transplant_doctor_2_queue.Mean())
    transplant_doctor_3_queue_length_avg.append(transplant_doctor_3_queue.Mean())

    q_flow_station_busy_avg.append(q_flow_station.CurrentNumBusy)
    secretary_station_busy_avg.append(secretary_station.CurrentNumBusy)
    nurse_station_1_busy_avg.append(nurse_station_1.CurrentNumBusy)
    nurse_station_2_busy_avg.append(nurse_station_2.CurrentNumBusy)
    nurse_station_3_busy_avg.append(nurse_station_3.CurrentNumBusy)
    nurse_station_4_busy_avg.append(nurse_station_4.CurrentNumBusy)
    nurse_station_5_busy_avg.append(nurse_station_5.CurrentNumBusy)
    nurse_station_6_busy_avg.append(nurse_station_6.CurrentNumBusy)
    leukemia_doctor_1_busy_avg.append(leukemia_doctor_1.CurrentNumBusy)
    leukemia_doctor_2_busy_avg.append(leukemia_doctor_2.CurrentNumBusy)
    transplant_doctor_1_busy_avg.append(transplant_doctor_1.CurrentNumBusy)
    transplant_doctor_2_busy_avg.append(transplant_doctor_2.CurrentNumBusy)
    transplant_doctor_3_busy_avg.append(transplant_doctor_3.CurrentNumBusy)

    pbar_outer.set_description(f"Running Simulation - {epoch+1}/{simulation_configuration.num_epochs}")
    pbar_outer.update(1)

pbar_outer.close()
logging.info("Simulation completed")

In [349]:
logging.info("Saving Simulation Results")
results_df = pd.DataFrame({
    "q_flow_station_wait_time_avg": q_flow_station_wait_time_avg,
    "q_flow_station_queue_length_avg": q_flow_station_queue_length_avg,
    "q_flow_station_busy_avg": q_flow_station_busy_avg,
    "secretary_station_wait_time_avg": secretary_station_wait_time_avg,
    "secretary_station_queue_length_avg": secretary_station_queue_length_avg,
    "secretary_station_busy_avg": secretary_station_busy_avg,
    "nurse_station_1_wait_time_avg": nurse_station_1_wait_time_avg,
    "nurse_station_1_queue_length_avg": nurse_station_1_queue_length_avg,
    "nurse_station_1_busy_avg": nurse_station_1_busy_avg,
    "nurse_station_2_wait_time_avg": nurse_station_2_wait_time_avg,
    "nurse_station_2_queue_length_avg": nurse_station_2_queue_length_avg,
    "nurse_station_2_busy_avg": nurse_station_2_busy_avg,
    "nurse_station_3_wait_time_avg": nurse_station_3_wait_time_avg,
    "nurse_station_3_queue_length_avg": nurse_station_3_queue_length_avg,
    "nurse_station_3_busy_avg": nurse_station_3_busy_avg,
    "nurse_station_4_wait_time_avg": nurse_station_4_wait_time_avg,
    "nurse_station_4_queue_length_avg": nurse_station_4_queue_length_avg,
    "nurse_station_4_busy_avg": nurse_station_4_busy_avg,
    "nurse_station_5_wait_time_avg": nurse_station_5_wait_time_avg,
    "nurse_station_5_queue_length_avg": nurse_station_5_queue_length_avg,
    "nurse_station_5_busy_avg": nurse_station_5_busy_avg,
    "nurse_station_6_wait_time_avg": nurse_station_6_wait_time_avg,
    "nurse_station_6_queue_length_avg": nurse_station_6_queue_length_avg,
    "nurse_station_6_busy_avg": nurse_station_6_busy_avg,
    "leukemia_doctor_1_wait_time_avg": leukemia_doctor_1_wait_time_avg,
    "leukemia_doctor_1_queue_length_avg": leukemia_doctor_1_queue_length_avg,
    "leukemia_doctor_1_busy_avg": leukemia_doctor_1_busy_avg,
    "leukemia_doctor_2_wait_time_avg": leukemia_doctor_2_wait_time_avg,
    "leukemia_doctor_2_queue_length_avg": leukemia_doctor_2_queue_length_avg,
    "leukemia_doctor_2_busy_avg": leukemia_doctor_2_busy_avg,
    "transplant_doctor_1_wait_time_avg": transplant_doctor_1_wait_time_avg,
    "transplant_doctor_1_queue_length_avg": transplant_doctor_1_queue_length_avg,
    "transplant_doctor_1_busy_avg": transplant_doctor_1_busy_avg,
    "transplant_doctor_2_wait_time_avg": transplant_doctor_2_wait_time_avg,
    "transplant_doctor_2_queue_length_avg": transplant_doctor_2_queue_length_avg,
    "transplant_doctor_2_busy_avg": transplant_doctor_2_busy_avg,
    "transplant_doctor_3_wait_time_avg": transplant_doctor_3_wait_time_avg,
    "transplant_doctor_3_queue_length_avg": transplant_doctor_3_queue_length_avg,
    "transplant_doctor_3_busy_avg": transplant_doctor_3_busy_avg,
    "other_patients_total_processing_time_avg": other_patients_total_processing_time_avg,
    "leukemia_doctor_1_complex_patients_total_processing_time_avg": leukemia_doctor_1_complex_patients_total_processing_time_avg,
    "leukemia_doctor_1_regular_patients_total_processing_time_avg": leukemia_doctor_1_regular_patients_total_processing_time_avg,
    "leukemia_doctor_2_complex_patients_total_processing_time_avg": leukemia_doctor_2_complex_patients_total_processing_time_avg,
    "leukemia_doctor_2_regular_patients_total_processing_time_avg": leukemia_doctor_2_regular_patients_total_processing_time_avg,
    "transplant_doctor_1_complex_patients_total_processing_time_avg": transplant_doctor_1_complex_patients_total_processing_time_avg,
    "transplant_doctor_1_regular_patients_total_processing_time_avg": transplant_doctor_1_regular_patients_total_processing_time_avg,
    "transplant_doctor_2_complex_patients_total_processing_time_avg": transplant_doctor_2_complex_patients_total_processing_time_avg,
    "transplant_doctor_2_regular_patients_total_processing_time_avg": transplant_doctor_2_regular_patients_total_processing_time_avg,
    "transplant_doctor_3_complex_patients_total_processing_time_avg": transplant_doctor_3_complex_patients_total_processing_time_avg,
    "transplant_doctor_3_regular_patients_total_processing_time_avg": transplant_doctor_3_regular_patients_total_processing_time_avg,
    })
os.makedirs("results_directory/simulation_multi_queue", exist_ok=True)
results_df.to_csv(f"results_directory/simulation_multi_queue/results_df.csv")
print(f"Simulation Results Saved to results_directory/simulation_multi_queue/results_df.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(results_df.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(BrooklynResultsDF)-1)*np.sqrt(BrooklynResultsDF.var()/len(BrooklynResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/results_df.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 5 minutes for all measures:")
print(max((scs.t.ppf(1-0.01, len(results_df)-1)*np.sqrt(results_df.var()/len(results_df))/5)**2))
print("Number of epochs required for 99% confidence, half-width 10 minutes for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(results_df)-1)*np.sqrt(results_df.var()/len(results_df))/10)**2))

Simulation Results Saved to BrooklynResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                     0.000659
Language Assessment Average Queue Length                  0.010900
Language Assessors Average Utilization                    0.393790
Adult Level A Course Average Wait Time                   10.235814
Adult Level A Course Average Queue Length                38.880208
Teachers Adult Level A Course Average Utilization         0.237840
Adult Level B Course Average Wait Time                   60.260987
Adult Level B Course Average Queue Length               358.630263
Teachers Adult Level B Course Average Utilization         0.613085
Adult Level C Course Average Wait Time                  162.250443
Adult Level C Course Average Queue Length               790.071156
Teachers Adult Level C Course Average Utilization         0.834283
Children Level A Course A

In [350]:
ManhattanResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": ManhattanLanguageAssessorsWaitTimeavg,
    "Language Assessment Average Queue Length": ManhattanLanguageAssessorsQueueLengthavg,
    "Language Assessors Average Utilization": np.array(ManhattanLanguageAssessorsBusyavg)/ManhattanNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": ManhattanTeachersAdultLevelAWaitTimeavg,
    "Adult Level A Course Average Queue Length": ManhattanTeachersAdultLevelAQueueLengthavg,
    "Teachers Adult Level A Course Average Utilization": np.array(ManhattanTeachersAdultLevelABusyavg)/ManhattanNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": ManhattanTeachersAdultLevelBWaitTimeavg,
    "Adult Level B Course Average Queue Length": ManhattanTeachersAdultLevelBQueueLengthavg,
    "Teachers Adult Level B Course Average Utilization": np.array(ManhattanTeachersAdultLevelBBusyavg)/ManhattanNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": ManhattanTeachersAdultLevelCWaitTimeavg,
    "Adult Level C Course Average Queue Length": ManhattanTeachersAdultLevelCQueueLengthavg,
    "Teachers Adult Level C Course Average Utilization": np.array(ManhattanTeachersAdultLevelCBusyavg)/ManhattanNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": ManhattanTeachersChildLevelAWaitTimeavg,
    "Children Level A Course Average Queue Length": ManhattanTeachersChildLevelAQueueLengthavg,
    "Teachers Children Level A Course Average Utilization": np.array(ManhattanTeachersChildLevelABusyavg)/ManhattanNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": ManhattanTeachersChildLevelBWaitTimeavg,
    "Children Level B Course Average Queue Length": ManhattanTeachersChildLevelBQueueLengthavg,
    "Teachers Children Level B Course Average Utilization": np.array(ManhattanTeachersChildLevelBBusyavg)/ManhattanNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": ManhattanTeachersChildLevelCWaitTimeavg,
    "Children Level C Course Average Queue Length": ManhattanTeachersChildLevelCQueueLengthavg,
    "Teachers Children Level C Course Average Utilization": np.array(ManhattanTeachersChildLevelCBusyavg)/ManhattanNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": ManhattanCareerCoachWaitTimeavg,
    "Career Guidance Average Queue Length": ManhattanCareerCoachQueueLengthavg,
    "Career Guides Average Utilization": np.array(ManhattanCareerCoachBusyavg)/ManhattanNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": ManhattanFluentChildrenTotalProcessTimeavg,
    "Level A Children Total Processing Time": ManhattanLevelAChildrenTotalProcessTimeavg,
    "Level B Children Total Processing Time": ManhattanLevelBChildrenTotalProcessTimeavg,
    "Level C Children Total Processing Time": ManhattanLevelCChildrenTotalProcessTimeavg,
    "Fluent Educated Adult Total Processing Time": ManhattanFluentEducatedAdultTotalProcessTimeavg,
    "Level A Educated Adult Total Processing Time": ManhattanLevelAEducatedAdultTotalProcessTimeavg,
    "Level B Educated Adult Total Processing Time": ManhattanLevelBEducatedAdultTotalProcessTimeavg,
    "Level C Educated Adult Total Processing Time": ManhattanLevelCEducatedAdultTotalProcessTimeavg,
    "Fluent Uneducated Adult Total Processing Time": ManhattanFluentUneducatedAdultTotalProcessTimeavg,
    "Level A Uneducated Adult Total Processing Time": ManhattanLevelAUneducatedAdultTotalProcessTimeavg,
    "Level B Uneducated Adult Total Processing Time": ManhattanLevelBUneducatedAdultTotalProcessTimeavg,
    "Level C Uneducated Adult Total Processing Time": ManhattanLevelCUneducatedAdultTotalProcessTimeavg,
    "Fluent Old Total Processing Time": ManhattanFluentOldTotalProcessTimeavg,
    "Level A Old Total Processing Time": ManhattanLevelAOldTotalProcessTimeavg,
    "Level B Old Total Processing Time": ManhattanLevelBOldTotalProcessTimeavg,
    "Level C Old Total Processing Time": ManhattanLevelCOldTotalProcessTimeavg,
    "Total Average Processing Time": ManhattanTotalProcessTimeavg
})
ManhattanResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/ManhattanResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to ManhattanResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(ManhattanResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(ManhattanResultsDF)-1)*np.sqrt(ManhattanResultsDF.var()/len(ManhattanResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/ManhattanResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(ManhattanResultsDF)-1)*np.sqrt(ManhattanResultsDF.var()/len(ManhattanResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(ManhattanResultsDF)-1)*np.sqrt(ManhattanResultsDF.var()/len(ManhattanResultsDF))/16)**2))

Simulation Results Saved to ManhattanResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                     0.009996
Language Assessment Average Queue Length                  0.079537
Language Assessors Average Utilization                    0.442010
Adult Level A Course Average Wait Time                   14.645042
Adult Level A Course Average Queue Length                26.762577
Teachers Adult Level A Course Average Utilization         0.312418
Adult Level B Course Average Wait Time                   63.001859
Adult Level B Course Average Queue Length               180.106784
Teachers Adult Level B Course Average Utilization         0.633394
Adult Level C Course Average Wait Time                  165.506694
Adult Level C Course Average Queue Length               386.873803
Teachers Adult Level C Course Average Utilization         0.846420
Children Level A Course 

In [351]:
QueensResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": QueensLanguageAssessorsWaitTimeavg,
    "Language Assessment Average Queue Length": QueensLanguageAssessorsQueueLengthavg,
    "Language Assessors Average Utilization": np.array(QueensLanguageAssessorsBusyavg)/QueensNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": QueensTeachersAdultLevelAWaitTimeavg,
    "Adult Level A Course Average Queue Length": QueensTeachersAdultLevelAQueueLengthavg,
    "Teachers Adult Level A Course Average Utilization": np.array(QueensTeachersAdultLevelABusyavg)/QueensNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": QueensTeachersAdultLevelBWaitTimeavg,
    "Adult Level B Course Average Queue Length": QueensTeachersAdultLevelBQueueLengthavg,
    "Teachers Adult Level B Course Average Utilization": np.array(QueensTeachersAdultLevelBBusyavg)/QueensNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": QueensTeachersAdultLevelCWaitTimeavg,
    "Adult Level C Course Average Queue Length": QueensTeachersAdultLevelCQueueLengthavg,
    "Teachers Adult Level C Course Average Utilization": np.array(QueensTeachersAdultLevelCBusyavg)/QueensNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": QueensTeachersChildLevelAWaitTimeavg,
    "Children Level A Course Average Queue Length": QueensTeachersChildLevelAQueueLengthavg,
    "Teachers Children Level A Course Average Utilization": np.array(QueensTeachersChildLevelABusyavg)/QueensNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": QueensTeachersChildLevelBWaitTimeavg,
    "Children Level B Course Average Queue Length": QueensTeachersChildLevelBQueueLengthavg,
    "Teachers Children Level B Course Average Utilization": np.array(QueensTeachersChildLevelBBusyavg)/QueensNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": QueensTeachersChildLevelCWaitTimeavg,
    "Children Level C Course Average Queue Length": QueensTeachersChildLevelCQueueLengthavg,
    "Teachers Children Level C Course Average Utilization": np.array(QueensTeachersChildLevelCBusyavg)/QueensNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": QueensCareerCoachWaitTimeavg,
    "Career Guidance Average Queue Length": QueensCareerCoachQueueLengthavg,
    "Career Guides Average Utilization": np.array(QueensCareerCoachBusyavg)/QueensNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": QueensFluentChildrenTotalProcessTimeavg,
    "Level A Children Total Processing Time": QueensLevelAChildrenTotalProcessTimeavg,
    "Level B Children Total Processing Time": QueensLevelBChildrenTotalProcessTimeavg,
    "Level C Children Total Processing Time": QueensLevelCChildrenTotalProcessTimeavg,
    "Fluent Educated Adult Total Processing Time": QueensFluentEducatedAdultTotalProcessTimeavg,
    "Level A Educated Adult Total Processing Time": QueensLevelAEducatedAdultTotalProcessTimeavg,
    "Level B Educated Adult Total Processing Time": QueensLevelBEducatedAdultTotalProcessTimeavg,
    "Level C Educated Adult Total Processing Time": QueensLevelCEducatedAdultTotalProcessTimeavg,
    "Fluent Uneducated Adult Total Processing Time": QueensFluentUneducatedAdultTotalProcessTimeavg,
    "Level A Uneducated Adult Total Processing Time": QueensLevelAUneducatedAdultTotalProcessTimeavg,
    "Level B Uneducated Adult Total Processing Time": QueensLevelBUneducatedAdultTotalProcessTimeavg,
    "Level C Uneducated Adult Total Processing Time": QueensLevelCUneducatedAdultTotalProcessTimeavg,
    "Fluent Old Total Processing Time": QueensFluentOldTotalProcessTimeavg,
    "Level A Old Total Processing Time": QueensLevelAOldTotalProcessTimeavg,
    "Level B Old Total Processing Time": QueensLevelBOldTotalProcessTimeavg,
    "Level C Old Total Processing Time": QueensLevelCOldTotalProcessTimeavg,
    "Total Average Processing Time": QueensTotalProcessTimeavg
})
QueensResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/QueensResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to QueensResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(QueensResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(QueensResultsDF)-1)*np.sqrt(QueensResultsDF.var()/len(QueensResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/QueensResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(QueensResultsDF)-1)*np.sqrt(QueensResultsDF.var()/len(QueensResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(QueensResultsDF)-1)*np.sqrt(QueensResultsDF.var()/len(QueensResultsDF))/16)**2))

Simulation Results Saved to QueensResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                     0.000399
Language Assessment Average Queue Length                  0.007488
Language Assessors Average Utilization                    0.390705
Adult Level A Course Average Wait Time                    9.304376
Adult Level A Course Average Queue Length                40.020120
Teachers Adult Level A Course Average Utilization         0.220614
Adult Level B Course Average Wait Time                   62.592887
Adult Level B Course Average Queue Length               422.259075
Teachers Adult Level B Course Average Utilization         0.624921
Adult Level C Course Average Wait Time                  161.977810
Adult Level C Course Average Queue Length               893.316486
Teachers Adult Level C Course Average Utilization         0.835722
Children Level A Course Ave

In [352]:
BronxResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": BronxLanguageAssessorsWaitTimeavg,
    "Language Assessment Average Queue Length": BronxLanguageAssessorsQueueLengthavg,
    "Language Assessors Average Utilization": np.array(BronxLanguageAssessorsBusyavg)/BronxNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": BronxTeachersAdultLevelAWaitTimeavg,
    "Adult Level A Course Average Queue Length": BronxTeachersAdultLevelAQueueLengthavg,
    "Teachers Adult Level A Course Average Utilization": np.array(BronxTeachersAdultLevelABusyavg)/BronxNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": BronxTeachersAdultLevelBWaitTimeavg,
    "Adult Level B Course Average Queue Length": BronxTeachersAdultLevelBQueueLengthavg,
    "Teachers Adult Level B Course Average Utilization": np.array(BronxTeachersAdultLevelBBusyavg)/BronxNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": BronxTeachersAdultLevelCWaitTimeavg,
    "Adult Level C Course Average Queue Length": BronxTeachersAdultLevelCQueueLengthavg,
    "Teachers Adult Level C Course Average Utilization": np.array(BronxTeachersAdultLevelCBusyavg)/BronxNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": BronxTeachersChildLevelAWaitTimeavg,
    "Children Level A Course Average Queue Length": BronxTeachersChildLevelAQueueLengthavg,
    "Teachers Children Level A Course Average Utilization": np.array(BronxTeachersChildLevelABusyavg)/BronxNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": BronxTeachersChildLevelBWaitTimeavg,
    "Children Level B Course Average Queue Length": BronxTeachersChildLevelBQueueLengthavg,
    "Teachers Children Level B Course Average Utilization": np.array(BronxTeachersChildLevelBBusyavg)/BronxNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": BronxTeachersChildLevelCWaitTimeavg,
    "Children Level C Course Average Queue Length": BronxTeachersChildLevelCQueueLengthavg,
    "Teachers Children Level C Course Average Utilization": np.array(BronxTeachersChildLevelCBusyavg)/BronxNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": BronxCareerCoachWaitTimeavg,
    "Career Guidance Average Queue Length": BronxCareerCoachQueueLengthavg,
    "Career Guides Average Utilization": np.array(BronxCareerCoachBusyavg)/BronxNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": BronxFluentChildrenTotalProcessTimeavg,
    "Level A Children Total Processing Time": BronxLevelAChildrenTotalProcessTimeavg,
    "Level B Children Total Processing Time": BronxLevelBChildrenTotalProcessTimeavg,
    "Level C Children Total Processing Time": BronxLevelCChildrenTotalProcessTimeavg,
    "Fluent Educated Adult Total Processing Time": BronxFluentEducatedAdultTotalProcessTimeavg,
    "Level A Educated Adult Total Processing Time": BronxLevelAEducatedAdultTotalProcessTimeavg,
    "Level B Educated Adult Total Processing Time": BronxLevelBEducatedAdultTotalProcessTimeavg,
    "Level C Educated Adult Total Processing Time": BronxLevelCEducatedAdultTotalProcessTimeavg,
    "Fluent Uneducated Adult Total Processing Time": BronxFluentUneducatedAdultTotalProcessTimeavg,
    "Level A Uneducated Adult Total Processing Time": BronxLevelAUneducatedAdultTotalProcessTimeavg,
    "Level B Uneducated Adult Total Processing Time": BronxLevelBUneducatedAdultTotalProcessTimeavg,
    "Level C Uneducated Adult Total Processing Time": BronxLevelCUneducatedAdultTotalProcessTimeavg,
    "Fluent Old Total Processing Time": BronxFluentOldTotalProcessTimeavg,
    "Level A Old Total Processing Time": BronxLevelAOldTotalProcessTimeavg,
    "Level B Old Total Processing Time": BronxLevelBOldTotalProcessTimeavg,
    "Level C Old Total Processing Time": BronxLevelCOldTotalProcessTimeavg,
    "Total Average Processing Time": BronxTotalProcessTimeavg
})
BronxResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/BronxResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to BronxResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(BronxResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(BronxResultsDF)-1)*np.sqrt(BronxResultsDF.var()/len(BronxResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/BronxResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(BronxResultsDF)-1)*np.sqrt(BronxResultsDF.var()/len(BronxResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(BronxResultsDF)-1)*np.sqrt(BronxResultsDF.var()/len(BronxResultsDF))/16)**2))

Simulation Results Saved to BronxResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                     0.002554
Language Assessment Average Queue Length                  0.021761
Language Assessors Average Utilization                    0.354975
Adult Level A Course Average Wait Time                   11.830166
Adult Level A Course Average Queue Length                23.169756
Teachers Adult Level A Course Average Utilization         0.244305
Adult Level B Course Average Wait Time                   68.547243
Adult Level B Course Average Queue Length               209.607294
Teachers Adult Level B Course Average Utilization         0.661430
Adult Level C Course Average Wait Time                  171.340042
Adult Level C Course Average Queue Length               429.654118
Teachers Adult Level C Course Average Utilization         0.855947
Children Level A Course Aver

In [353]:
StatenIslandResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": StatenIslandLanguageAssessorsWaitTimeavg,
    "Language Assessment Average Queue Length": StatenIslandLanguageAssessorsQueueLengthavg,
    "Language Assessors Average Utilization": np.array(StatenIslandLanguageAssessorsBusyavg)/StatenIslandNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": StatenIslandTeachersAdultLevelAWaitTimeavg,
    "Adult Level A Course Average Queue Length": StatenIslandTeachersAdultLevelAQueueLengthavg,
    "Teachers Adult Level A Course Average Utilization": np.array(StatenIslandTeachersAdultLevelABusyavg)/StatenIslandNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": StatenIslandTeachersAdultLevelBWaitTimeavg,
    "Adult Level B Course Average Queue Length": StatenIslandTeachersAdultLevelBQueueLengthavg,
    "Teachers Adult Level B Course Average Utilization": np.array(StatenIslandTeachersAdultLevelBBusyavg)/StatenIslandNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": StatenIslandTeachersAdultLevelCWaitTimeavg,
    "Adult Level C Course Average Queue Length": StatenIslandTeachersAdultLevelCQueueLengthavg,
    "Teachers Adult Level C Course Average Utilization": np.array(StatenIslandTeachersAdultLevelCBusyavg)/StatenIslandNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": StatenIslandTeachersChildLevelAWaitTimeavg,
    "Children Level A Course Average Queue Length": StatenIslandTeachersChildLevelAQueueLengthavg,
    "Teachers Children Level A Course Average Utilization": np.array(StatenIslandTeachersChildLevelABusyavg)/StatenIslandNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": StatenIslandTeachersChildLevelBWaitTimeavg,
    "Children Level B Course Average Queue Length": StatenIslandTeachersChildLevelBQueueLengthavg,
    "Teachers Children Level B Course Average Utilization": np.array(StatenIslandTeachersChildLevelBBusyavg)/StatenIslandNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": StatenIslandTeachersChildLevelCWaitTimeavg,
    "Children Level C Course Average Queue Length": StatenIslandTeachersChildLevelCQueueLengthavg,
    "Teachers Children Level C Course Average Utilization": np.array(StatenIslandTeachersChildLevelCBusyavg)/StatenIslandNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": StatenIslandCareerCoachWaitTimeavg,
    "Career Guidance Average Queue Length": StatenIslandCareerCoachQueueLengthavg,
    "Career Guides Average Utilization": np.array(StatenIslandCareerCoachBusyavg)/StatenIslandNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": StatenIslandFluentChildrenTotalProcessTimeavg,
    "Level A Children Total Processing Time": StatenIslandLevelAChildrenTotalProcessTimeavg,
    "Level B Children Total Processing Time": StatenIslandLevelBChildrenTotalProcessTimeavg,
    "Level C Children Total Processing Time": StatenIslandLevelCChildrenTotalProcessTimeavg,
    "Fluent Educated Adult Total Processing Time": StatenIslandFluentEducatedAdultTotalProcessTimeavg,
    "Level A Educated Adult Total Processing Time": StatenIslandLevelAEducatedAdultTotalProcessTimeavg,
    "Level B Educated Adult Total Processing Time": StatenIslandLevelBEducatedAdultTotalProcessTimeavg,
    "Level C Educated Adult Total Processing Time": StatenIslandLevelCEducatedAdultTotalProcessTimeavg,
    "Fluent Uneducated Adult Total Processing Time": StatenIslandFluentUneducatedAdultTotalProcessTimeavg,
    "Level A Uneducated Adult Total Processing Time": StatenIslandLevelAUneducatedAdultTotalProcessTimeavg,
    "Level B Uneducated Adult Total Processing Time": StatenIslandLevelBUneducatedAdultTotalProcessTimeavg,
    "Level C Uneducated Adult Total Processing Time": StatenIslandLevelCUneducatedAdultTotalProcessTimeavg,
    "Fluent Old Total Processing Time": StatenIslandFluentOldTotalProcessTimeavg,
    "Level A Old Total Processing Time": StatenIslandLevelAOldTotalProcessTimeavg,
    "Level B Old Total Processing Time": StatenIslandLevelBOldTotalProcessTimeavg,
    "Level C Old Total Processing Time": StatenIslandLevelCOldTotalProcessTimeavg,
    "Total Average Processing Time": StatenIslandTotalProcessTimeavg
})
StatenIslandResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/StatenIslandResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to StatenIslandResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(StatenIslandResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(StatenIslandResultsDF)-1)*np.sqrt(StatenIslandResultsDF.var()/len(StatenIslandResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/StatenIslandResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(StatenIslandResultsDF)-1)*np.sqrt(StatenIslandResultsDF.var()/len(StatenIslandResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(StatenIslandResultsDF)-1)*np.sqrt(StatenIslandResultsDF.var()/len(StatenIslandResultsDF))/16)**2))

Simulation Results Saved to StatenIslandResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                      0.033907
Language Assessment Average Queue Length                   0.058252
Language Assessors Average Utilization                     0.286257
Adult Level A Course Average Wait Time                    44.993793
Adult Level A Course Average Queue Length                 17.705445
Teachers Adult Level A Course Average Utilization          0.511018
Adult Level B Course Average Wait Time                   124.740944
Adult Level B Course Average Queue Length                 76.187190
Teachers Adult Level B Course Average Utilization          0.854827
Adult Level C Course Average Wait Time                   215.263505
Adult Level C Course Average Queue Length                105.586253
Teachers Adult Level C Course Average Utilization          0.896572
Children 

In [354]:
Brooklyn_N = max((scs.t.ppf(1-0.01/2, len(BrooklynResultsDF)-1)*np.sqrt(BrooklynResultsDF.var()/len(BrooklynResultsDF))/16**2))
Manhattan_N = max((scs.t.ppf(1-0.01/2, len(ManhattanResultsDF)-1)*np.sqrt(ManhattanResultsDF.var()/len(ManhattanResultsDF))/16)**2)
Queens_N = max((scs.t.ppf(1-0.01/2, len(QueensResultsDF)-1)*np.sqrt(QueensResultsDF.var()/len(QueensResultsDF))/16)**2)
Bronx_N = max((scs.t.ppf(1-0.01/2, len(BronxResultsDF)-1)*np.sqrt(BronxResultsDF.var()/len(BronxResultsDF))/16)**2)
StatenIsland_N = max((scs.t.ppf(1-0.01/2, len(StatenIslandResultsDF)-1)*np.sqrt(StatenIslandResultsDF.var()/len(StatenIslandResultsDF))/16)**2)

print(f"Number of epochs required for 99% confidence, half-width 16 (two days) for all measures: {math.ceil(max(Brooklyn_N, Manhattan_N, Queens_N, Bronx_N, StatenIsland_N))}")

Number of epochs required for 99% confidence, half-width 16 (two days) for all measures: 99



# According to a single pilot (5 epochs) we would require 65 iterations to achieve confidence intervals with confidence level of 99\% and half width of 16 (2 days) for all tracked statistics


In [355]:
QueueLengthsDict = {
    "BrooklynLanguageAssessors": BrooklynLanguageAssessorsQueueLength,
    "BrooklynTeachersAdultLevelA": BrooklynTeachersAdultLevelAQueueLength,
    "BrooklynTeachersAdultLevelB": BrooklynTeachersAdultLevelBQueueLength,
    "BrooklynTeachersAdultLevelC": BrooklynTeachersAdultLevelCQueueLength,
    "BrooklynTeachersChildLevelA": BrooklynTeachersChildLevelAQueueLength,
    "BrooklynTeachersChildLevelB": BrooklynTeachersChildLevelBQueueLength,
    "BrooklynTeachersChildLevelC": BrooklynTeachersChildLevelCQueueLength,
    "BrooklynCareerGuides": BrooklynCareerCoachesQueueLength,
    "ManhattanLanguageAssessors": ManhattanLanguageAssessorsQueueLength,
    "ManhattanTeachersAdultLevelA": ManhattanTeachersAdultLevelAQueueLength,
    "ManhattanTeachersAdultLevelB": ManhattanTeachersAdultLevelBQueueLength,
    "ManhattanTeachersAdultLevelC": ManhattanTeachersAdultLevelCQueueLength,
    "ManhattanTeachersChildLevelA": ManhattanTeachersChildLevelAQueueLength,
    "ManhattanTeachersChildLevelB": ManhattanTeachersChildLevelBQueueLength,
    "ManhattanTeachersChildLevelC": ManhattanTeachersChildLevelCQueueLength,
    "ManhattanCareerGuides": ManhattanCareerCoachesQueueLength,
    "QueensLanguageAssessors": QueensLanguageAssessorsQueueLength,
    "QueensTeachersAdultLevelA": QueensTeachersAdultLevelAQueueLength,
    "QueensTeachersAdultLevelB": QueensTeachersAdultLevelBQueueLength,
    "QueensTeachersAdultLevelC": QueensTeachersAdultLevelCQueueLength,
    "QueensTeachersChildLevelA": QueensTeachersChildLevelAQueueLength,
    "QueensTeachersChildLevelB": QueensTeachersChildLevelBQueueLength,
    "QueensTeachersChildLevelC": QueensTeachersChildLevelCQueueLength,
    "QueensCareerGuides": QueensCareerCoachesQueueLength,
    "BronxLanguageAssessors": BronxLanguageAssessorsQueueLength,
    "BronxTeachersAdultLevelA": BronxTeachersAdultLevelAQueueLength,
    "BronxTeachersAdultLevelB": BronxTeachersAdultLevelBQueueLength,
    "BronxTeachersAdultLevelC": BronxTeachersAdultLevelCQueueLength,
    "BronxTeachersChildLevelA": BronxTeachersChildLevelAQueueLength,
    "BronxTeachersChildLevelB": BronxTeachersChildLevelBQueueLength,
    "BronxTeachersChildLevelC": BronxTeachersChildLevelCQueueLength,
    "BronxCareerGuides": BronxCareerCoachesQueueLength,
    "StatenIslandLanguageAssessors": StatenIslandLanguageAssessorsQueueLength,
    "StatenIslandTeachersAdultLevelA": StatenIslandTeachersAdultLevelAQueueLength,
    "StatenIslandTeachersAdultLevelB": StatenIslandTeachersAdultLevelBQueueLength,
    "StatenIslandTeachersAdultLevelC": StatenIslandTeachersAdultLevelCQueueLength,
    "StatenIslandTeachersChildLevelA": StatenIslandTeachersChildLevelAQueueLength,
    "StatenIslandTeachersChildLevelB": StatenIslandTeachersChildLevelBQueueLength,
    "StatenIslandTeachersChildLevelC": StatenIslandTeachersChildLevelCQueueLength,
    "StatenIslandCareerGuides": StatenIslandCareerCoachesQueueLength
}
#Adding zeros to force into df. didnt have time to find a better alternative:(
max_length = max(len(arr) for arr in QueueLengthsDict.values())
for key, value in QueueLengthsDict.items():
    QueueLengthsDict[key] += [0] * (max_length - len(value))
QueueLengthsDataFrame = pd.DataFrame(QueueLengthsDict)
QueueLengthsDataFrame.to_csv(f"ResultsDirectory/SimulationUncooperative/QueueLengthsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to QueueLengthsMode{Mode}_{k}.csv")

Simulation Results Saved to QueueLengthsMode4_6.csv


In [356]:
Immigrants_successfully_processed_avg_df = pd.DataFrame(Immigrants_successfully_processed_avg, columns = ['Children', 'Adults', 'Old'])
Immigrants_successfully_processed_avg_df.to_csv(f"ResultsDirectory/SimulationUncooperative/ImmigrantsProcessedMode{Mode}_{k}.csv", sep=",")